# Title: Champions League Data Scrape and Analysis
## Author: Martin Ngoh 
### Date: 05/10/2024

In [112]:
# Load Libraries
import pandas as pd
import numpy as np 
import re

In [113]:
# Load Data 
df = pd.read_csv('champs.csv')

In [114]:
# Head of Df 
df.head(2)

,Stage,Round,Group,Date,Team 1,FT,HT,Team 2,∑FT,ET,P,Comments
0,Qualifying,Qual. Round 1 | Leg 1,NaN,(Tue) 15 Jul 2008 (W28),Inter Baku PIK › AZE (1),0-0,0-0,Rabotnicki › MKD (1),NaN,NaN,NaN,NaN
1,Qualifying,Qual. Round 1 | Leg 1,NaN,(Tue) 15 Jul 2008 (W28),Anorthosis Famagusta › CYP (1),1-0,0-0,Pyunik › ARM (1),NaN,NaN,NaN,NaN


In [115]:
# Info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Stage     213 non-null    object
 1   Round     213 non-null    object
 2   Group     96 non-null     object
 3   Date      213 non-null    object
 4   Team 1    213 non-null    object
 5   FT        213 non-null    object
 6   HT        213 non-null    object
 7   Team 2    213 non-null    object
 8   ∑FT       58 non-null     object
 9   ET        2 non-null      object
 10  P         1 non-null      object
 11  Comments  4 non-null      object
dtypes: object(12)
memory usage: 20.1+ KB


In [116]:
# Subset the Columns 
df = df[['Stage', 'Date', 'Team 1', 'Team 2', 'FT', 'HT']]

In [117]:
# Head of DF 
df.head(2)

,Stage,Date,Team 1,Team 2,FT,HT
0,Qualifying,(Tue) 15 Jul 2008 (W28),Inter Baku PIK › AZE (1),Rabotnicki › MKD (1),0-0,0-0
1,Qualifying,(Tue) 15 Jul 2008 (W28),Anorthosis Famagusta › CYP (1),Pyunik › ARM (1),1-0,0-0


## Date Column 

In [118]:
# Do Some Processing of the Date Column. 
df['Date'] = df['Date'].apply(lambda x: re.sub(r'\(W[1-9][0-9]?\)', '', x))
df['Date'] = df['Date'].apply(lambda x: re.sub(r'\((Tue|Wed)\)', '', x))

In [119]:
# Now Look at the date column 
df['Date'][:2]

0     15 Jul 2008 
1     15 Jul 2008 
Name: Date, dtype: object

In [120]:
# Transform the Column into Numeric Date 
df['Date'] = pd.to_datetime(df['Date'])
df['Date'][:1]

0   2008-07-15
Name: Date, dtype: datetime64[ns]

In [121]:
# Look at df 
df.head(2)

,Stage,Date,Team 1,Team 2,FT,HT
0,Qualifying,2008-07-15,Inter Baku PIK › AZE (1),Rabotnicki › MKD (1),0-0,0-0
1,Qualifying,2008-07-15,Anorthosis Famagusta › CYP (1),Pyunik › ARM (1),1-0,0-0


## Stage Column

In [122]:
# Print Stage Values
df['Stage'].value_counts()

Stage
Group         96
Qualifying    88
Knockout      29
Name: count, dtype: int64

In [123]:
# Do Mappings for Values 
stage_map = {
    'Group':1, 
    'Qualifying':2,
    'Knockout':3
}
df['Stage'] = df['Stage'].map(stage_map)

In [124]:
# Look at Stage Column 
df['Stage'].value_counts()

Stage
1    96
2    88
3    29
Name: count, dtype: int64

In [125]:
# Look at df 
df.head(2)

,Stage,Date,Team 1,Team 2,FT,HT
0,2,2008-07-15,Inter Baku PIK › AZE (1),Rabotnicki › MKD (1),0-0,0-0
1,2,2008-07-15,Anorthosis Famagusta › CYP (1),Pyunik › ARM (1),1-0,0-0


## Team 1 and Team 1 

In [126]:
# Print team1 and 2 
df['Team 1'][:1]

0    Inter Baku PIK › AZE (1)
Name: Team 1, dtype: object

In [127]:
# Remove Last 10 characters
df['Team 1'] = df['Team 1'].apply(lambda x: x[:-10])
df['Team 2'] = df['Team 2'].apply(lambda x: x[:-10])


## Look at the FT and HT Columns 

In [128]:
# FT 
df['FT'][:2]

0    0-0
1    1-0
Name: FT, dtype: object

In [129]:
# Split them up to create fulltime team 1 score, full time team 2 score 
df['ft_t1_gls'] = df['FT'].apply(lambda x: x[0] if isinstance(x, str) and len(x) > 0 else x)
df['ft_t2_gls'] = df['FT'].apply(lambda x: x[-1] if isinstance(x, str) and len(x) > 0 else x)

In [130]:
# Do the Same for the HT
df['ht_t1_gls'] = df['HT'].apply(lambda x: int(x[0]) if isinstance(x, str) and len(x) > 0 else x)
df['ht_t2_gls'] = df['HT'].apply(lambda x: int(x[-1]) if isinstance(x, str) and len(x) > 0 else x)

In [131]:
# Look at results
df[1:5]

,Stage,Date,Team 1,Team 2,FT,HT,ft_t1_gls,ft_t2_gls,ht_t1_gls,ht_t2_gls
1,2,2008-07-15,Anorthosis Famagusta,Pyunik,1-0,0-0,1,0,0,0
2,2,2008-07-15,BATE Borisov,Valur Reykjavik,2-0,0-0,2,0,0,0
3,2,2008-07-15,Tampere United,Buducnost,2-1,0-0,2,1,0,0
4,2,2008-07-15,Dinamo Tirana,FK Modrica,0-2,0-0,0,2,0,0


## Look At Results

In [132]:
# Print DF 
df.head(2)

,Stage,Date,Team 1,Team 2,FT,HT,ft_t1_gls,ft_t2_gls,ht_t1_gls,ht_t2_gls
0,2,2008-07-15,Inter Baku PIK,Rabotnicki,0-0,0-0,0,0,0,0
1,2,2008-07-15,Anorthosis Famagusta,Pyunik,1-0,0-0,1,0,0,0


In [133]:
# Save out 
df = df.drop(['FT', 'HT'], axis = 1)
df.to_csv('champs_cleaned.csv', index = False)

# Process Team Data 

In [134]:
# Pull in Team data 
team = pd.read_csv('team_data.csv')

In [135]:
# Drop NAs
team = team.dropna(axis = 1)
# rename squad ]
team.rename(columns = {'Squad':'Team'}, inplace=True)

In [136]:
# Join the data 
team.to_csv('team_data_cleaned.csv', index = False)